## Named entity recognition

In [1]:
import spacy

In [2]:
nlp = spacy.load('pt_core_news_lg')

# Frase
doc = nlp(u'Rio de Janeiro é a capital turística do Brasil.')

In [3]:
# Print das Entidades e suas classes
print ("\n-------Exemplo 1 ------\n")
for ent in doc.ents:
    print(ent.label_, ent.text)


-------Exemplo 1 ------

LOC Rio de Janeiro
LOC Brasil


In [4]:
# Frase
doc1 = nlp(u'Enquanto isso na França, Christine Lagarde discutiu os esforços de estímulo de curto prazo em uma '
           u'entrevista recente às 17:00 com o Wall Street Journal')

# Print das Entidades e suas classes
print ("\n-------Exemplo 2 ------\n")
for ent1 in doc1.ents:
    print(ent1.label_, ent1.text)


-------Exemplo 2 ------

LOC França
PER Christine Lagarde
MISC Wall Street Journal


In [5]:
doc = nlp("Enquanto isso na França, Christine Lagarde discutiu os esforços de estímulo de curto prazo em uma entrevista recente às 17:00 com o Wall Street Journal")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

Enquanto enquanto ADP ADP case Xxxxx True True
isso isso PRON PRON obl xxxx True True
na em o ADP ADP case xx True True
França França PROPN PROPN nmod Xxxxx True False
, , PUNCT PUNCT punct , False False
Christine Christine PROPN PROPN nsubj Xxxxx True False
Lagarde Lagarde PROPN PROPN flat:name Xxxxx True False
discutiu discutir VERB VERB ROOT xxxx True False
os o DET DET det xx True True
esforços esforço NOUN NOUN obj xxxx True False
de de ADP ADP case xx True True
estímulo estímulo NOUN NOUN nmod xxxx True False
de de ADP ADP case xx True True
curto curto ADJ ADJ amod xxxx True False
prazo prazo NOUN NOUN nmod xxxx True False
em em ADP ADP case xx True True
uma um DET DET det xxx True True
entrevista entrevista NOUN NOUN obl xxxx True False
recente recente ADJ ADJ amod xxxx True False
às a o ADP ADP case xx True True
17:00 17:00 NOUN NOUN nmod dd:dd False False
com com ADP ADP case xxx True True
o o DET DET det x True True
Wall Wall PROPN PROPN nmod Xxxx True False
Street Street PRO

In [7]:
print("Pipeline:", nlp.pipe_names)

Pipeline: ['tok2vec', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


In [10]:
doc = nlp("Eu estava lendo o artigo.")
token = doc[0]
print(token.morph)
print(token.morph.get("PronType")) 

Case=Nom|Gender=Fem|Number=Sing|Person=1|PronType=Prs
['Prs']


In [12]:
# Imports
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Cria um ngram vectorizer
# Vamos criar o BOW de 2 documentos, que neste caso serão as palavras: words e wprds
# Usamos n_gram = 2 no nível de unidade de caracter
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2), min_df=1)

# Contabilizamos as combinações de caracteres nas palavras
counts = ngram_vectorizer.fit_transform(['words', 'wprds'])
ngram_vectorizer.get_feature_names_out() == (['w', 'ds', 'or', 'pr', 'rd', 's ', 'wo', 'wp'])

# Imprimimos o BOW
print (counts.toarray().astype(int))


[[1 1 1 0 1 1 1 0]
 [1 1 0 1 1 1 0 1]]


In [13]:
# pip install textblob

# Imports
from textblob import TextBlob
import math

# TF
def tf(word, blob):
       return blob.words.count(word) / len(blob.words)

# Verifica palavra mno texto
def n_containing(word, bloblist):
    return 1 + sum(1 for blob in bloblist if word in blob)

# IDF
def idf(word, bloblist):
    x = n_containing(word, bloblist)
    return math.log(len(bloblist) / (x if x else 1))

# Score TF-IDF
def tfidf(word, blob, bloblist):
   return tf(word, blob) * idf(word, bloblist)

# Texto
text = 'tf idf, forma abreviada de frequência de termo, frequência de documento inversa'
text2 = 'é uma estatística numérica que se destina a refletir o quão importante'
text3 = 'uma palavra é para um documento em uma coleção ou corpus'

blob = TextBlob(text)
blob2 = TextBlob(text2)
blob3 = TextBlob(text3)
bloblist = [blob, blob2, blob3]

# Scores para a palavra: abreviada
tf_score = tf('abreviada', blob)
idf_score = idf('abreviada', bloblist)
tfidf_score = tfidf('abreviada', blob, bloblist)

# Print
print ("\n")
print ("TF Score para a palavra: abreviada     --- " + str(tf_score)+"\n")
print ("IDF Score para a palavra: abreviada    --- " + str(idf_score)+"\n")
print ("TF-IDF Score para a palavra: abreviada --- " + str(tfidf_score))
print ("\n")



TF Score para a palavra: abreviada     --- 0.08333333333333333

IDF Score para a palavra: abreviada    --- 0.4054651081081644

TF-IDF Score para a palavra: abreviada --- 0.033788759009013694




In [14]:
# Aplicando TF-IDF em um conjunto de dados
# http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

# Imports
import os
import nltk
import string
import numpy as np
from nltk.stem.porter import *
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Função para obter os tokens
def get_tokens():
   with open('shakes/shakes1.txt', 'r') as shakes:
    text = shakes.read()
    lowers = text.lower()

    # Remove a pontuação usando a etapa de exclusão de caracteres da função translate()
    no_punctuation = lowers.translate(string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

# Obtém os tokens
tokens = get_tokens()
print("\nTokens: ", tokens)

# Remove as Stop words
filtered = [w for w in tokens if not w in stopwords.words('english')]
print("\nTokens sem stop words: ", tokens)

# Função para Stem
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

# Aplica o Stem
stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)
print("\nTokens sem stop words e com stemming: ", stemmed)

# Variáveis de automatização
path = 'shakes'
token_dict = {}

# Tokenização
def tokenize(text):
    no_punctuation = lowers.translate(string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    stems = stem_tokens(tokens, stemmer)
    return stems

# Obtém todos os arquivos no Corpus e remove a pontuação
for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        shakes = open(file_path, 'r')
        text = shakes.read()
        lowers = text.lower()
        no_punctuation = lowers.translate(string.punctuation)
        token_dict[file] = no_punctuation

# Calcula o TF-IDF
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

print("\nFeatures")
feature_names = tfidf.get_feature_names()
for col in tfs.nonzero()[1]:
    print (feature_names[col], ' - ', tfs[0, col])


Tokens:  ['this', 'young', 'gentlewoman', 'had', 'a', 'father', ',', '--', 'o', ',', "that+'had", "'", '!', 'how', 'sad', 'a', 'passage', "'t", 'is', '!', '--', 'whose', 'skill', 'was+almost', 'as', 'great', 'as', 'his', 'honesty', ';', 'had', 'it', 'stretched', 'so+far', ',', 'would', 'have', 'made', 'nature', 'immortal', ',', 'and', 'death+should', 'have', 'play', 'for', 'lack', 'of', 'work', '.', 'would', ',', 'for', 'the+king', "'s", 'sake', ',', 'he', 'were', 'living', '!', 'i', 'think', 'it', 'would', 'be+the', 'death', 'of', 'the', 'king', "'s", 'disease.+he', 'was', 'excellent', 'indeed', ',', 'madam', ':', 'the', 'king', 'very+lately', 'spoke', 'of', 'him', 'admiringly', 'and', 'mourningly', ':', 'he+was', 'skilful', 'enough', 'to', 'have', 'lived', 'still', ',', 'if', 'knowledge+could', 'be', 'set', 'up', 'against', 'mortality.+his', 'sole', 'child', ',', 'my', 'lord', ',', 'and', 'bequeathed', 'to', 'my+overlooking', '.', 'i', 'have', 'those', 'hopes', 'of', 'her', 'good', 


Tokens sem stop words:  ['this', 'young', 'gentlewoman', 'had', 'a', 'father', ',', '--', 'o', ',', "that+'had", "'", '!', 'how', 'sad', 'a', 'passage', "'t", 'is', '!', '--', 'whose', 'skill', 'was+almost', 'as', 'great', 'as', 'his', 'honesty', ';', 'had', 'it', 'stretched', 'so+far', ',', 'would', 'have', 'made', 'nature', 'immortal', ',', 'and', 'death+should', 'have', 'play', 'for', 'lack', 'of', 'work', '.', 'would', ',', 'for', 'the+king', "'s", 'sake', ',', 'he', 'were', 'living', '!', 'i', 'think', 'it', 'would', 'be+the', 'death', 'of', 'the', 'king', "'s", 'disease.+he', 'was', 'excellent', 'indeed', ',', 'madam', ':', 'the', 'king', 'very+lately', 'spoke', 'of', 'him', 'admiringly', 'and', 'mourningly', ':', 'he+was', 'skilful', 'enough', 'to', 'have', 'lived', 'still', ',', 'if', 'knowledge+could', 'be', 'set', 'up', 'against', 'mortality.+his', 'sole', 'child', ',', 'my', 'lord', ',', 'and', 'bequeathed', 'to', 'my+overlooking', '.', 'i', 'have', 'those', 'hopes', 'of', 


Features
+  -  0.09205746178983235
valu  -  0.09205746178983235
time  -  0.09205746178983235
young  -  0.09205746178983235
remors  -  0.09205746178983235
pleasur  -  0.09205746178983235
wa  -  0.1841149235796647
+it  -  0.09205746178983235
stay  -  0.09205746178983235
entreat  -  0.09205746178983235
did  -  0.09205746178983235
you.+i  -  0.09205746178983235
knowledg  -  0.09205746178983235
love  -  0.09205746178983235
desir  -  0.09205746178983235
shall  -  0.09205746178983235
+i  -  0.1841149235796647
world  -  0.09205746178983235
better  -  0.09205746178983235
+hereaft  -  0.09205746178983235
:  -  0.1841149235796647
fare  -  0.09205746178983235
sir  -  0.09205746178983235
.  -  0.09205746178983235
forth  -  0.09205746178983235
break  -  0.09205746178983235
suddenli  -  0.09205746178983235
lady+wil  -  0.09205746178983235
malic  -  0.09205746178983235
life  -  0.09205746178983235
+and  -  0.09205746178983235
;  -  0.276172385369497
sake  -  0.09205746178983235
's  -  0.0920574617898

C:\Users\Rodrigo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'gainst", "'s", '+', '+and', '+ground', '+hereaft', '+i', '+it', ',', '.', ':', ';', 'argument+but', 'better', 'bond', 'break', 'dearer', 'desir', 'did', 'displeasur', 'duke+hath', 'entreat', 'fare', 'father', 'forth', 'gentl', 'good', 'hi', 'knowledg', 'lady+wil', 'late', 'life', 'love', 'malic', 'natur', 'niec', 'peopl', 'piti', 'pleasur', 'prais', 'remors', 'sake', 'shall', 'sir', 'sisters.+but', 'stay', 'suddenli', "ta'en", 'tell', 'thi', 'time', 'valu', 'virtues+and', 'wa', 'world', 'you.+i', 'young'] not in stop_words.
  warnings.warn(
